In [4]:
# data comes from http://web.stanford.edu/~ericyi/project_page/part_annotation/index.html
from pyntcloud import PyntCloud
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
import numpy as np

import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/PartAnnotation"
else:
    data_path = os.getcwd() + "\\PartAnnotation"

# Load model

In [103]:
def get_all_var_names():
    all_vars = []
    for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
        all_vars.append(i.name)
    return all_vars

def get_all_placeholders():
    return [x for x in tf.get_default_graph().get_operations() if x.type == "Placeholder"]

def get_all_mean_op():
    return [x for x in tf.get_default_graph().get_operations() if x.type == "Mean"]

def remove_var(var_name, var_set):
    if var_name in var_set:
        var_set.remove(var_name)

In [65]:
new_graph = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True) # allow passing gpu-trained model to a cpu machine
with tf.Session(graph=new_graph, config=config) as sess:

    import os
    model_fqn = os.path.join(os.getcwd(), 'trained_model', "model-2")
    saver = tf.train.import_meta_graph(model_fqn + ".meta")
    saver.restore(sess, model_fqn)

    graph = tf.get_default_graph()
    print(get_all_var_names())
    print("\nOptimizer", graph.get_collection("optimizer"))
    print("\nPlaceholders", get_all_placeholders())
    print("\nMean", get_all_mean_op())
    previous_vars = {var.op.name for var in tf.global_variables()}
#     previous_vars = [var_name for var_name, _ in tf.contrib.framework.list_variables('ckpt')]
    print(previous_vars)

INFO:tensorflow:Restoring parameters from C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2


INFO:tensorflow:Restoring parameters from C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2


['conv1/kernel:0', 'conv1/bias:0', 'conv2/kernel:0', 'conv2/bias:0', 'conv4/kernel:0', 'conv4/bias:0', 'conv5/kernel:0', 'conv5/bias:0', 'conv7/kernel:0', 'conv7/bias:0', 'conv8/kernel:0', 'conv8/bias:0', 'bn/gamma:0', 'bn/beta:0', 'bn/moving_mean:0', 'bn/moving_variance:0', 'full_con/kernel:0', 'full_con/bias:0', 'y_pred/kernel:0', 'y_pred/bias:0', 'training/beta1_power:0', 'training/beta2_power:0', 'conv1/kernel/Adam:0', 'conv1/kernel/Adam_1:0', 'conv1/bias/Adam:0', 'conv1/bias/Adam_1:0', 'conv2/kernel/Adam:0', 'conv2/kernel/Adam_1:0', 'conv2/bias/Adam:0', 'conv2/bias/Adam_1:0', 'conv4/kernel/Adam:0', 'conv4/kernel/Adam_1:0', 'conv4/bias/Adam:0', 'conv4/bias/Adam_1:0', 'conv5/kernel/Adam:0', 'conv5/kernel/Adam_1:0', 'conv5/bias/Adam:0', 'conv5/bias/Adam_1:0', 'conv7/kernel/Adam:0', 'conv7/kernel/Adam_1:0', 'conv7/bias/Adam:0', 'conv7/bias/Adam_1:0', 'conv8/kernel/Adam:0', 'conv8/kernel/Adam_1:0', 'conv8/bias/Adam:0', 'conv8/bias/Adam_1:0', 'bn/gamma/Adam:0', 'bn/gamma/Adam_1:0', 'bn/

In [7]:
def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs. (We will only take x,y,z into account for now)
    dim: dimensioin of output voxelized data
    
    This function will locate the grid cube and calculate the density of each cube.
    The output will be normalized values.
    """
    assert(pts.shape[1]>=3), "pts file should contain at least x,y,z coordinate"
    assert(len(dim)==3), "Please provide 3-d grid size like [32,32,32]"
    
    # move all the axis to positive area.
    minimum_val = [pts[0][0], pts[0][1], pts[0][2]]

    # find the smallest 
    for pair in pts:
        if pair[0] < minimum_val[0]:
            minimum_val[0] = pair[0]
        if pair[1] < minimum_val[1]:
            minimum_val[1] = pair[1]
        if pair[2] < minimum_val[2]:
            minimum_val[2] = pair[2]
            
    # move it to first quadrant 
    rectified_pts = np.empty(pts.shape)
    for index, pair in enumerate(pts):
        point = np.zeros(3)
        point[0] = pair[0] - minimum_val[0]
        point[1] = pair[1] - minimum_val[1]
        point[2] = pair[2] - minimum_val[2]
        rectified_pts[index] = point
    
    # biggest value in each axis 
    maximum_val = pts[0][0]
    
    for pair in rectified_pts:
        for val in pair:
            if val > maximum_val:
                maximum_val = val
     
    # normalize all the axises to (0,1)
    normalized_pts = rectified_pts/maximum_val
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2]))
    
    epsilon = 0.000000000001 # we will have at least a 1.0 value which will exceed the index of grid
    # we can use a relativly small value to escape that to fit our data
    
    max_volume_size = 0
    
    for pair in normalized_pts:
        x_loc = int(pair[0]/(x_grid_length + epsilon))
        y_loc = int(pair[1]/(y_grid_length + epsilon))
        z_loc = int(pair[2]/(z_grid_length + epsilon))
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
        
        if output[x_loc, y_loc, z_loc] > max_volume_size:
            max_volume_size = output[x_loc, y_loc, z_loc]
    
    output = output/max_volume_size    
            
    return output

In [8]:
def get_data(data_path, max_file_num=None, dim=[32,32,32]):
    data = []
    
    target_dir_path = os.path.join(data_path, 'points')
    path, dirs, files = os.walk(target_dir_path).__next__()
    file_count = len(files)
    
    count = 0
    for pts_data in os.scandir(target_dir_path):
        if (max_file_num is None) or (count < max_file_num):
            _path = os.path.join(data_path, 'points', pts_data.name)
            pts = PyntCloud.from_file(_path, sep=" ", header=0, names=["x","y","z"])
            _vox = voxelize3D(pts.xyz, dim=dim)
            vox_chan = np.array(_vox).reshape(_vox.shape + (1,))
            data.append(vox_chan)
            count += 1
        else:
            break
                
    return data

In [9]:
def get_model_params(model_path):
    
    new_graph = tf.Graph()
    config = tf.ConfigProto(allow_soft_placement=True) # allow passing gpu-trained model to a cpu machine
    
    with tf.Session(graph=new_graph, config=config) as sess:
        
        # Restore model
        saver = tf.train.import_meta_graph(model_path + ".meta")
        saver.restore(sess, model_path)

        model_graph = tf.get_default_graph()
        
        _conv1_w = model_graph.get_tensor_by_name('conv1/kernel:0')
        _conv1_b = model_graph.get_tensor_by_name('conv1/bias:0')   

        _conv2_w = model_graph.get_tensor_by_name('conv2/kernel:0')
        _conv2_b = model_graph.get_tensor_by_name('conv2/bias:0')  

        _conv4_w = model_graph.get_tensor_by_name('conv4/kernel:0')
        _conv4_b = model_graph.get_tensor_by_name('conv4/bias:0')

        _conv5_w = model_graph.get_tensor_by_name('conv5/kernel:0')
        _conv5_b = model_graph.get_tensor_by_name('conv5/bias:0')
        
        _conv7_w = model_graph.get_tensor_by_name('conv7/kernel:0')
        _conv7_b = model_graph.get_tensor_by_name('conv7/bias:0')
        
        _conv8_w = model_graph.get_tensor_by_name('conv8/kernel:0')
        _conv8_b = model_graph.get_tensor_by_name('conv8/bias:0')
    
        _dict = {
            'conv1_w': _conv1_w.eval(),
            'conv1_b': _conv1_b.eval(),
            'conv2_w': _conv2_w.eval(),
            'conv2_b': _conv2_b.eval(),
            'conv4_w': _conv4_w.eval(),
            'conv4_b': _conv4_b.eval(),
            'conv5_w': _conv5_w.eval(),
            'conv5_b': _conv5_b.eval(),
            'conv7_w': _conv7_w.eval(),
            'conv7_b': _conv7_b.eval(),
            'conv8_w': _conv8_w.eval(),
            'conv8_b': _conv8_b.eval(),
        }

    return _dict
    

In [20]:
import h5py
import os
h5_data = h5py.File(os.path.join(os.path.join(os.getcwd(), 'h5dataset'), 'big_shuffled_data.h5'), mode='r')

data = h5_data.get("voxels")
label = h5_data.get("labels")
label_ref = h5_data.get("label_ref")

print(data.shape, label.shape, label_ref.value)

(29147, 32, 32, 32, 1) (29147, 5) [b'car' b'lamp' b'table' b'airplane' b'chair']


In [96]:
import tensorflow as tf
def cnn3d_model(x_train_data, label_size, keep_rate=0.7, seed=None):
    
    if seed is not None:
        tf.set_random_seed(seed)
    
    with tf.name_scope("layer_a"):
        # conv => 32*32*32
        conv1 = tf.layers.conv3d(inputs=x_train_data, filters=16, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv1", reuse=tf.AUTO_REUSE)
        # conv => 32*32*32
        conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv2", reuse=tf.AUTO_REUSE)
        # pool => 16*16*16
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2, name="pool3")
        
    with tf.name_scope("layer_b"):
        # conv => 16*16*16
        conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv4", reuse=tf.AUTO_REUSE)
        # conv => 16*16*16
        conv5 = tf.layers.conv3d(inputs=conv4, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv5", reuse=tf.AUTO_REUSE)
        # pool => 8*8*8
        pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2, name="pool6")
        
    with tf.name_scope("layer_c"):
        # conv => 8*8*8
        conv7 = tf.layers.conv3d(inputs=pool6, filters=256, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv7", reuse=tf.AUTO_REUSE)
        # conv => 8*8*8
        conv8 = tf.layers.conv3d(inputs=conv7, filters=512, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv8", reuse=tf.AUTO_REUSE)
        # pool => 4*4*4
        pool9 = tf.layers.max_pooling3d(inputs=conv8, pool_size=[2, 2, 2], strides=2, name="pool9")
        
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=pool9, training=True, name="bn", reuse=tf.AUTO_REUSE)
        
    with tf.name_scope("fully_con"):
        flattening = tf.reshape(cnn3d_bn, [-1, 4*4*4*512])
        dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu, name="full_con", reuse=tf.AUTO_REUSE)
        # (1-keep_rate) is the probability that the node will be kept
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True, name="dropout")
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=label_size, name="y_pred", reuse=tf.AUTO_REUSE)
    
    return y_conv

In [108]:
# one hot indexes

import os
# model_path = os.path.join(os.getcwd(), 'trained_model', 'model-2')

# params = get_model_params(model_path)

learning_rate = 0.001

file_num =1000

x_guitar = get_data(data_path + "/guitar", max_file_num=file_num) 
x_ = x_guitar[:500]
y_ = np.zeros((500, 6))
# x_ = data[:1000]
# y_ = label[:1000]

for index, _ in enumerate(y_):
    y_[index][5] = 1
        
split_point = int(0.7*len(x_))
x_train = x_[:split_point]
y_train = y_[:split_point]

x_test = x_[split_point:]
y_test = y_[split_point:]

n_classes = 6

device_name = '/gpu:1' 

with tf.Session(graph=tf.Graph()) as sess:
    with tf.device(device_name):
        with tf.name_scope('inputs'):
            x_input = tf.placeholder(tf.float32, shape=[None, 32, 32, 32, 1], name="x_input")
            y_input = tf.placeholder(tf.float32, shape=[None, n_classes], name="y_input") 

        prediction = cnn3d_model(x_input, n_classes, seed=1234, keep_rate=0)
        tf.add_to_collection("logits", prediction)

        with tf.name_scope("cross_entropy"):
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input), name="cross_entropy")

        with tf.name_scope("training"):
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
            tf.add_to_collection("optimizer", optimizer)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="acc")
        
    print(tf.trainable_variables())
    remove_var('y_pred/kernel',previous_vars)
    remove_var('y_pred/bias',previous_vars)
    remove_var('y_pred/kernel/Adam',previous_vars)
    remove_var('y_pred/bias/Adam',previous_vars)
    remove_var('y_pred/kernel/Adam_1',previous_vars)
    remove_var('y_pred/bias/Adam_1',previous_vars)
    
    restore_map = {variable.op.name : variable for variable in tf.global_variables() if variable.op.name in previous_vars}
    tf.contrib.framework.init_from_checkpoint(os.path.join(os.getcwd(), 'trained_model', 'model-2'), restore_map)
    
    sess.run(tf.global_variables_initializer())
    
    # stop gradient here
    
    graph = tf.get_default_graph()
    
    
    batch_size = 32
    iterations_all = int(len(x_)/batch_size) +1
    iterations_train = int(len(x_train)/batch_size) +1
    iterations_test= int(len(x_test)/batch_size) +1
    
    acc = 0
    
    for itr in range(iterations_all):
        mini_batch_x = x_[itr*batch_size: (itr+1)*batch_size]
        mini_batch_y = y_[itr*batch_size: (itr+1)*batch_size]
        _acc = sess.run(accuracy, feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
        acc += _acc
        print('\tLost for', itr+1, "/", iterations_all, _acc, end='\r')
    
    print('\n')
    print('Test Acc:', acc/iterations_all)
    
    for itr in range(iterations_train):
        mini_batch_x = x_train[itr*batch_size: (itr+1)*batch_size]
        mini_batch_y = y_train[itr*batch_size: (itr+1)*batch_size]
        _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
        print('\tLost for', itr+1, "/", iterations_train, _cost, end='\r')
    
    print('\n')
    
    acc = 0
    for itr in range(iterations_all):
        mini_batch_x = x_[itr*batch_size: (itr+1)*batch_size]
        mini_batch_y = y_[itr*batch_size: (itr+1)*batch_size]
        _acc = sess.run(accuracy, feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
        acc += _acc
        print('\tLost for', itr+1, "/", iterations_all, end='\r')
    
    print('\n')
    print('Test Acc:', acc/iterations_all)

[<tf.Variable 'conv1/kernel:0' shape=(3, 3, 3, 1, 16) dtype=float32_ref>, <tf.Variable 'conv1/bias:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'conv2/kernel:0' shape=(3, 3, 3, 16, 32) dtype=float32_ref>, <tf.Variable 'conv2/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'conv4/kernel:0' shape=(3, 3, 3, 32, 64) dtype=float32_ref>, <tf.Variable 'conv4/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv5/kernel:0' shape=(3, 3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'conv5/bias:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'conv7/kernel:0' shape=(3, 3, 3, 128, 256) dtype=float32_ref>, <tf.Variable 'conv7/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'conv8/kernel:0' shape=(3, 3, 3, 256, 512) dtype=float32_ref>, <tf.Variable 'conv8/bias:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'bn/gamma:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'bn/beta:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'full_con/kernel:0' shape=(32768, 1024) dtype=float32_ref>

INFO:tensorflow:Initialize variable conv1/kernel:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with conv1/kernel


INFO:tensorflow:Initialize variable conv1/bias:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with conv1/bias


INFO:tensorflow:Initialize variable conv1/bias:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with conv1/bias


INFO:tensorflow:Initialize variable conv2/kernel:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with conv2/kernel


...

INFO:tensorflow:Initialize variable full_con/bias/Adam:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with full_con/bias/Adam


INFO:tensorflow:Initialize variable full_con/bias/Adam:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with full_con/bias/Adam


INFO:tensorflow:Initialize variable full_con/bias/Adam_1:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with full_con/bias/Adam_1


INFO:tensorflow:Initialize variable full_con/bias/Adam_1:0 from checkpoint C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2 with full_con/bias/Adam_1


	Lost for 16 / 16 0.55755

Test Acc: 0.610546875745
	Lost for 11 / 11 0.0564e-08

	Lost for 16 / 16

Test Acc: 1.0


In [24]:
# one hot indexes
# 2: chair , 3: car

import os
model_path = os.path.join(os.getcwd(), 'trained_model', 'model-0')

params = get_model_params(model_path)

file_num =7000

a = get_data(data_path + "/airplane", max_file_num=file_num)
    
device_name = '/gpu:1' 

with tf.Session(graph=tf.Graph()) as sess:
    with tf.device(device_name):
        with tf.name_scope('inputs'):
            x_input = tf.placeholder(tf.float32, shape=[None, 32, 32, 32, 1], name="x_input")
            y_input = tf.placeholder(tf.float32, shape=[None, 5], name="y_input") 

        prediction = cnn3d_model_with_default(params, x_input, 5, stop_layer=10, seed=1234)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name="acc")
    
    
    sess.run(tf.global_variables_initializer())
    
    graph = tf.get_default_graph()
    
    batch_size = 32
    accu = []
    iterations_test = int(file_num/batch_size) +1
    for itr in range(iterations_test):
        mini_batch_x = a[itr*batch_size: (itr+1)*batch_size]
        mini_batch_y = y[itr*batch_size: (itr+1)*batch_size]
#         acc = sess.run(accuracy, feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
        if len(mini_batch_x) == 0:
            continue
        p = np.argmax(sess.run(prediction, feed_dict={x_input: mini_batch_x}), 1)
        accu.append(p)
        print('\taccuracy for', itr+1, "/", iterations_train, p, end='\r')
    
    print('\n')
    
    pppp = {}
    for _ in range(len(accu)):
        for i,v in enumerate(accu[_]):
            if accu[_][i] in pppp:
                pppp[v] = pppp[v] + 1
            else:
                pppp[v] = 1

INFO:tensorflow:Restoring parameters from C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-0


INFO:tensorflow:Restoring parameters from C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-0


NameError: name 'y' is not defined

In [63]:
p_table

{0: 1343, 1: 476, 2: 139, 3: 2684, 4: 2346, 5: 12}

In [49]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tag_constants.TRAINING], model_fqn)

NameError: name 'tag_constants' is not defined

# Plot

In [268]:
# %load ./voxel_grid_plot.py
import os
import shutil

import numpy as np

from IPython.display import IFrame
from matplotlib import pyplot as plt


def plot_voxelgrid(voxelgrid,
                   output_name=None,
                   cmap="Oranges",
                   axis=True,
                   width=800,
                   height=600):

    scaled_shape = voxelgrid.shape

    vector = voxelgrid
    points = np.argwhere(vector) * scaled_shape

    s_m = plt.cm.ScalarMappable(cmap=cmap)
    rgb = s_m.to_rgba(vector.reshape(-1)[vector.reshape(-1) > 0])

    camera_position = points.max(0) + abs(points.max(0))

    look = points.mean(0)

    if axis:
        axis_size = points.ptp() * 1.5
    else:
        axis_size = 0

    placeholders = {}

    placeholders["POINTS_X_PLACEHOLDER"] = points[:, 0].tolist()
    placeholders["POINTS_Y_PLACEHOLDER"] = points[:, 1].tolist()
    placeholders["POINTS_Z_PLACEHOLDER"] = points[:, 2].tolist()

    placeholders["R_PLACEHOLDER"] = rgb[:, 0].tolist()
    placeholders["G_PLACEHOLDER"] = rgb[:, 1].tolist()
    placeholders["B_PLACEHOLDER"] = rgb[:, 2].tolist()

    placeholders["S_x_PLACEHOLDER"] = scaled_shape[0]
    placeholders["S_y_PLACEHOLDER"] = scaled_shape[1]
    placeholders["S_z_PLACEHOLDER"] = scaled_shape[2]

    placeholders["CAMERA_X_PLACEHOLDER"] = camera_position[0]
    placeholders["CAMERA_Y_PLACEHOLDER"] = camera_position[1]
    placeholders["CAMERA_Z_PLACEHOLDER"] = camera_position[2]

    placeholders["LOOK_X_PLACEHOLDER"] = look[0]
    placeholders["LOOK_Y_PLACEHOLDER"] = look[1]
    placeholders["LOOK_Z_PLACEHOLDER"] = look[2]

    placeholders["AXIS_SIZE_PLACEHOLDER"] = axis_size

    placeholders["N_VOXELS_PLACEHOLDER"] = sum(vector.reshape(-1) > 0)

    if output_name is None:
        output_name = "plotVG.html"

    BASE_PATH = os.getcwd()
    src = "{}/{}".format(BASE_PATH, "voxelgrid.html")
    dst = "{}/{}".format(os.getcwd(), output_name)

    with open(src, "r") as inp, open(dst, "w") as out:
        for line in inp:
            for key, val in placeholders.items():
                if key in line:
                    line = line.replace(key, str(val))
            out.write(line)

    return IFrame(output_name, width=width, height=height)

# Feature extraction

In [11]:
# one hot indexes
# 2: chair , 3: car

import os
model_path = os.path.join(os.getcwd(), 'trained_model', 'model-4')

params = get_model_params(model_path)

print(params['conv1_w'].shape)

INFO:tensorflow:Restoring parameters from /Users/shijian/git/3D-CNN/3d_pointcloud/trained_model/model-4
(3, 3, 3, 1, 16)


In [12]:
file_num =7000

a = get_data(data_path + "/airplane", max_file_num=file_num)

In [380]:
def do_padding(target, kernel, stride, padding='SAME'):
    # Algorithm comes from https://github.com/tensorflow/tensorflow/commit/a276e0999ab4223ac36d75221028d3e8835c60ae
    size, kernelsize, color_channels = get_shapes(target, kernel)
    
    import math
    if padding is 'SAME':
        out_width = math.ceil(float(size) / float(stride))
        pad_along_width = max((out_width - 1) * stride + kernelsize - size, 0)
        pad_left_and_top = pad_along_width // 2
        pad_right_and_bottom = pad_along_width - pad_left_and_top
        padded_target = zero_padding_3d(target, (int((pad_left_and_top)), int(pad_right_and_bottom)))
        
    elif padding is 'VALID':
        # no padding would be used
        out_width = math.floor(float(size - kernelsize) / float(stride) + 1)
        border = (out_width-1)*stride + kernelsize
        print(out_width, border)
        padded_target = target[:border,:border,:border, :]
        
    else:
        raise TypeError('Padding strategy `' + padding + '` not found.')
        
    print("Padded shape", padded_target.shape)
    return padded_target


def zero_padding_3d(target, padding=None):
    if padding is not None:
        target = np.pad(target, [(padding[0], padding[1]), (padding[0], padding[1]), (padding[0], padding[1]), (0,0)], mode='constant')
    return target


def single_convolve_3d(padded_target_slice, kernel):
    _slice = padded_target_slice * kernel
    res = np.sum(np.sum(np.sum(np.sum(_slice, axis=0), axis=0),axis=0), axis=0)
    return res


def convole_3d(padded_target, kernel, kernel_bias, stride):
    
    size, kernelsize, color_channels = get_shapes(padded_target, kernel)
    outsize = get_outsize(size, kernelsize, stride)
    filter_num = num_of_filters(kernel)
    
    res = np.zeros((outsize, outsize, outsize, filter_num))
    
    for i in range(outsize):
        for j in range(outsize):
            for k in range(outsize):
                for f in range(filter_num):
                    i_start = i * stride
                    j_start = j * stride
                    k_start = k * stride

                    padded_target_slice = padded_target[i_start : i_start+kernelsize, j_start : j_start+kernelsize, k_start : k_start+kernelsize, :]
                    res[i_start, j_start, k_start, f] = single_convolve_3d(padded_target_slice, kernel[:,:,:,:,f])
                    
    print(res.shape)
    return res + kernel_bias
   
def max_pool_slice(target_slice):
    return np.max(target_slice)

def get_shapes(target, kernel):
    size = target.shape[0]
    color_channels = target.shape[3]
    kernelsize = kernel.shape[0]
    assert(color_channels == kernel.shape[3]), "The shape of kernel and input does not match, it must have same number of channels."
    return size, kernelsize, color_channels

def get_outsize(size, kernelsize, stride):
    outsize = (size-kernelsize)/stride + 1
    assert(outsize == int(outsize)), "Number of filters must be integer"
    
    return int(outsize)

def num_of_filters(kernel):
    return kernel.shape[4]

In [381]:
def output_features(target, kernel, bias=None, stride=1, padding='SAME'):
    
    size, kernelsize, color_channels = get_shapes(target, kernel)
    outsize = get_outsize(size, kernelsize, stride)
    filter_num = num_of_filters(kernel)
    
    padded_target = do_padding(target, kernel, stride)
    output = convole_3d(padded_target, kernel, bias, stride)
    
    print(output.shape)
    return output


def max_pooling(target, pool_size=2, stride=2):
    
    size = target.shape[0]
    outsize = get_outsize(size, pool_size, stride)
    filter_num = target.shape[3]
    
    output = np.zeros((outsize, outsize, outsize, filter_num))
    for i in range(outsize):
        for j in range(outsize):
            for k in range(outsize):
                for f in range(filter_num):
                    i_start = i*stride
                    j_start = j*stride
                    k_start = k*stride
                    
                    output[i, j, k, f] = max_pool_slice(target[i_start:i_start+2, j_start:j_start+2, k_start:k_start+2, f])
                    
    return output


In [331]:
first_conv = output_features(a[0], params['conv1_w'], params['conv1_b'], stride=1, padding='SAME')
plot_voxelgrid(first_conv[:,:,:,1])

Padded shape (34, 34, 34, 1)
(32, 32, 32, 16)
(32, 32, 32, 16)


In [333]:
second_conv = output_features(first_conv, params['conv2_w'], params['conv2_b'], stride=1, padding='SAME')
plot_voxelgrid(second_conv[:,:,:,1])

Padded shape (34, 34, 34, 16)
(32, 32, 32, 32)
(32, 32, 32, 32)


In [386]:
third_pool = max_pooling(second_conv, pool_size=2, stride=2)
plot_voxelgrid(third_pool[:,:,:,1])

In [398]:
forth_conv = output_features(third_pool, params['conv4_w'], params['conv4_b'], stride=1, padding='SAME')
fifth_conv = output_features(forth_conv, params['conv5_w'], params['conv5_b'], stride=1, padding='SAME')
sixth_pooling = max_pooling(fifth_conv, pool_size=2, stride=2)

Padded shape (18, 18, 18, 32)
(16, 16, 16, 64)
(16, 16, 16, 64)
Padded shape (18, 18, 18, 64)
(16, 16, 16, 128)
(16, 16, 16, 128)


In [397]:
plot_voxelgrid(sixth_pooling[:,:,:,10])

In [399]:
seventh_conv = output_features(sixth_pooling, params['conv7_w'], params['conv7_b'], stride=1, padding='SAME')
eighth_conv = output_features(seventh_conv, params['conv8_w'], params['conv8_b'], stride=1, padding='SAME')
ninth_pooling = max_pooling(eighth_conv, pool_size=2, stride=2)

Padded shape (10, 10, 10, 128)
(8, 8, 8, 256)
(8, 8, 8, 256)
Padded shape (10, 10, 10, 256)
(8, 8, 8, 512)
(8, 8, 8, 512)


In [ ]:
plot_voxelgrid(sixth_pooling[:,:,:,10])